In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import timedelta

In [2]:
df = pd.read_csv('T10Y2Y.csv')

In [3]:
print(df.shape)
df.head()

(11437, 2)


,DATE,T10Y2Y
0,1976-06-01,0.68
1,1976-06-02,0.71
2,1976-06-03,0.70
3,1976-06-04,0.77
4,1976-06-07,0.79


In [4]:
df.dtypes

DATE      object
T10Y2Y    object
dtype: object

In [5]:
df = df[df['T10Y2Y'] != '.']

In [6]:
def convert_yield(row):
    if row != '.':
        return float(row)
    else: 
        return 0.00

In [7]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['T10Y2Y'] = df['T10Y2Y'].map(lambda x: float(x))

In [8]:
daily_yield = df.groupby(['DATE'])['T10Y2Y'].sum()
daily_yield = pd.DataFrame(daily_yield, columns=['T10Y2Y'])

In [9]:
daily_yield

,T10Y2Y
DATE,
1976-06-01,0.68
1976-06-02,0.71
1976-06-03,0.70
1976-06-04,0.77
1976-06-07,0.79
...,...
2020-03-26,0.53
2020-03-27,0.47
2020-03-30,0.47


In [10]:
start_dt = daily_yield.index[0]
start_dt

Timestamp('1976-06-01 00:00:00')

In [11]:
end_dt = daily_yield.index[-1]
end_dt

Timestamp('2020-04-01 00:00:00')

In [ ]:
f = go.Figure()
f.add_trace(go.Scatter(x=daily_yield.index, 
                       y=daily_yield.T10Y2Y, 
                       line_color='rgb(0, 0, 100)', 
                       ))

f.update_layout(title_text=
                '10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity')
f.add_shape(
        # Line Horizontal
            type="line",
            x0=start_dt,
            y0=0,
            x1=end_dt,
            y1=0,
            line=dict(
                color="red",
                width=2,
                
            ))

for index, row in daily_yield.iterrows():
    if row['T10Y2Y'] >= 0:
        f.add_shape(
            type= 'rect',
            # x-reference is assigned to the x-values
            xref= 'x',
            # y-reference is assigned to the plot paper [0,1]
            yref= 'paper',
            x0= str(index.normalize()),
            y0= 0,
            x1= str((index+timedelta(days=1)).normalize()),
            y1= 1,
            fillcolor= '#d3d3d3',
            opacity= 0.2,
            line= {
                'width': 0,
            }
        )       

f.show()